In [ ]:
!pip install --quiet zarr scikit-image

## Affine Transformations

Spatial transformations are defined per image layer via a [4x4 transformation matrix](https://www.brainvoyager.com/bv/doc/UsersGuide/CoordsAndTransforms/SpatialTransformationMatrices.html) using the `model_matrix` property.

This features is useful for pre-processing (e.g. flipping, rotating, scaling etc) your data, since these operations 
are executed performantly on the GPU thanks to [Viv](https://github.com/hms-dbmi/viv).

In this notebook we will load a simple OME-NGFF dataset from the `Getting Started` notebook and apply a simple rotation.

In [ ]:
from create_fixture import create_ome_zarr
import zarr
import numpy as np

import vizarr

# creates an example OME-NGFF, see `getting_started.ipynb` for more details
create_ome_zarr("astronaut.zarr") 

# Create a viewer
viewer = vizarr.Viewer()

astronaut = {
    "source": zarr.open("astronaut.zarr", mode="r"),
    "name": "astronaut",
}

noise = {
    "source": zarr.array(np.random.randint(0, 255, (1024, 1024), dtype=np.uint8)),
    "opacity": 0.8,
    "name": "noise",
}

viewer.add_image(**astronaut)
viewer.add_image(**noise)
viewer

### Apply translation and rotation to `noise` layer

In [ ]:
M = np.eye(4)

# rotation
theta = np.pi / 4
M[:3, :3] = np.array([
    [ np.cos(theta),  np.sin(theta), 0],
    [-np.sin(theta),  np.cos(theta), 0],
    [             0,              0, 1],
])

# translation
M[:3,3] = np.array([512, 512, 0])

M

In [ ]:
translated_noise = { **noise, **{"model_matrix": M.ravel().tolist() } } # need to ravel into 1D list

v = vizarr.Viewer()
v.add_image(**astronaut)
v.add_image(**translated_noise)
v